# Analyse Flow File

In [1]:
import json
from itertools import product

In [2]:
def find_number_of_generated_cars(flow):
    
    startTime = flow["startTime"]
    endTime = flow["endTime"]
    interval = flow["interval"]

    num_interval = (endTime - startTime + 1) // interval

    mod_interval = (endTime - startTime + 1) % interval != 0

    number_of_generated_cars = int(num_interval + mod_interval) 
    
    return number_of_generated_cars

In [3]:
def analyze(mapping, all_roads, all_combinations, flow_data, total_num_flows, total_num_flows_direct, total_num_vehicles):

    vehicles = []
    
    remaining_flows = []

    for flow in flow_data:
    
        num_generated_cars = find_number_of_generated_cars(flow)
    
        in_road = flow["route"][0]
        out_road = flow["route"][-1]

        found = False
        direct_found = False

        for in_road_type, out_road_type in all_combinations:
            
            all_in_roads = all_roads[in_road_type].keys()
            all_out_roads = all_roads[out_road_type].values()
            
            if in_road in all_in_roads and out_road in all_out_roads:
                
                total_num_flows[(in_road_type, mapping[out_road_type])] += 1
                total_num_vehicles[(in_road_type, mapping[out_road_type])] += num_generated_cars
                found = True

                if found:
                    if out_road in all_roads[in_road_type][in_road]:
                        total_num_flows_direct[(in_road_type, mapping[out_road_type])] += 1
                        direct_found = True
                        
                    
                if found:
                    break


        if not found:
            remaining_flows.append(flow)
            total_num_flows["Remaining"] += 1
            total_num_vehicles["Remaining"] += num_generated_cars

        if not direct_found:
            total_num_flows_direct["Remaining"] += 1
            
        
    return remaining_flows


In [7]:
mapping = {"A1": "A2", "A2": "A1", "A3": "A4", "A4": "A3", "D": "D", "B1": "B2", "B2": "B1", "C1": "C2", "C2": "C1"}

A1 = {f"road_0_{i}_0": f"road_30_{i}_0" for i in range(9,12)}
A2 = {f"road_31_{i}_2": f"road_1_{i}_2" for i in range(9,12)}
A3 = {f"road_0_{i}_0": f"road_30_{i}_0" for i in range(1,7)}
A4 = {f"road_31_{i}_2": f"road_1_{i}_2" for i in range(1,7)}

D = {f"road_0_{i}_0": f"road_1_{i}_2" for i in range(7,9)}

B1 = {f"road_{i}_12_3": f"road_{i}_1_3" for i in range(1, 11)}
B2 = {f"road_{i}_0_1": f"road_{i}_11_1" for i in range(1, 11)}
C1 = {f"road_{i}_12_3": f"road_{i}_1_3" for i in range(11, 31)}
C2 = {f"road_{i}_0_1": f"road_{i}_11_1" for i in range(11, 31)}


all_roads = {"A1": A1, "A2": A2, "A3": A3, "A4": A4, "D": D, "B1": B1, "B2": B2, "C1": C1, "C2": C2}


avenues = {**A1, **A2, **A3, **A4, **D}
streets = {**B1, **B2, **C1, **C2}
all_roads = {"Avenue": avenues, "Streets": streets}
mapping = {"Avenue": "Avenue", "Streets": "Streets"}

# all combinations we test
all_combinations = list(product(all_roads.keys(), all_roads.keys()))

total_num_flows = {key:0 for key in all_combinations}
total_num_flows_direct = {key:0 for key in all_combinations}
total_num_vehicles = {key:0 for key in all_combinations}

total_num_flows.update({"Remaining": 0})
total_num_flows_direct.update({"Remaining": 0})
total_num_vehicles.update({"Remaining": 0})


with open("../../Simulation_Results/Manhattan/flow.json",'r') as f:
    flow_data = json.load(f)


remaining_flows = analyze(mapping, all_roads, all_combinations, flow_data, total_num_flows, total_num_flows_direct, total_num_vehicles)           

assert sum(total_num_flows.values()) == len(flow_data), f"Don't match: {sum(total_num_flows.values())} and {len(flow_data)}"


number_routes = len(flow_data)
number_vehicles = sum(total_num_vehicles.values())
print(f"Total routes in flow file: {number_routes}")
print(f"Total number of generated vehicles: {number_vehicles}")
print()

next_ele = all_combinations[0][0]
for od_pair in all_combinations + ["Remaining"]:
    first_ele = od_pair[0]
    
    if first_ele != next_ele:
        print()
    
    percentage_cars = round((total_num_vehicles[od_pair]/number_vehicles) * 100,1)
    
    print(f"For OD pair: {od_pair}:{'':<2} Total different routes: {total_num_flows[od_pair]:<5} {'':<2}Total number of generated vehicles: {total_num_vehicles[od_pair]:<5} ({percentage_cars} %)")
    
    next_ele = first_ele
    
    
    

print()
print(f"Total different routes: Total: {sum(total_num_flows.values())}")
print(f"Total different routes (only straight): Total: {sum(total_num_flows_direct.values())}")
print(f"Total number of generated vehicles:  Total: {sum(total_num_vehicles.values())}")

Total routes in flow file: 1452
Total number of generated vehicles: 10073

For OD pair: ('Avenue', 'Avenue'):   Total different routes: 113     Total number of generated vehicles: 2640  (26.2 %)
For OD pair: ('Avenue', 'Streets'):   Total different routes: 379     Total number of generated vehicles: 2161  (21.5 %)

For OD pair: ('Streets', 'Avenue'):   Total different routes: 494     Total number of generated vehicles: 3388  (33.6 %)
For OD pair: ('Streets', 'Streets'):   Total different routes: 281     Total number of generated vehicles: 1144  (11.4 %)

For OD pair: Remaining:   Total different routes: 185     Total number of generated vehicles: 740   (7.3 %)

Total different routes: Total: 1452
Total different routes (only straight): Total: 1452
Total number of generated vehicles:  Total: 10073


In [ ]:
Total routes in flow file: 1452
Total number of generated vehicles: 10073

For OD pair: ('Avenue', 'Avenue'):   Total different routes: 113     Total number of generated vehicles: 2640  (26.2 %)
For OD pair: ('Avenue', 'Streets'):   Total different routes: 379     Total number of generated vehicles: 2161  (21.5 %)

For OD pair: ('Streets', 'Avenue'):   Total different routes: 494     Total number of generated vehicles: 3388  (33.6 %)
For OD pair: ('Streets', 'Streets'):   Total different routes: 281     Total number of generated vehicles: 1144  (11.4 %)

For OD pair: Remaining:   Total different routes: 185     Total number of generated vehicles: 740   (7.3 %)

Total different routes: Total: 1452
Total different routes (only straight): Total: 1452
Total number of generated vehicles:  Total: 10073

In [1]:
import json


A1_star = {f"road_1_{i}_0": f"road_29_{e}_0" for (i,e) in zip(range(1, 12), [1,2,3,4,5,6,6,9,9,10,11])}
A2_star = {f"road_30_{i}_2": f"road_2_{i}_2" for i in range(1, 12) if i not in {7, 8}}
B1_star = {f"road_{i}_11_3": f"road_{i}_2_3" for i in range(1, 11)}
B2_star = {f"road_{i}_1_1": f"road_{i}_10_1" for i in range(1, 11)}
C1_star = {f"road_{i}_11_3": f"road_{i}_2_3" for i in range(11, 31)}
C2_star = {f"road_{i}_1_1": f"road_{i}_10_1" for i in range(10, 31)}



with open("../Simulation_Results/Manhattan/flow_Penn.json",'r') as f:
    flow_data = json.load(f)

    

all_roads = {"A1_star": A1_star, "A2_star": A2_star, "B1_star": B1_star, "B2_star": B2_star, "C1_star": C1_star, "C2_star": C2_star}


change_in_flows = []
change_out_flows = []

for index, flow in enumerate(flow_data):
    in_road = flow["route"][0]
    out_road = flow["route"][-1]
    
    for route_type, roads in all_roads.items():
        if in_road in roads.keys():
            change_in_flows.append(index)
            
        if out_road in roads.values():
            change_out_flows.append(index)
        
            
            
print(change_in_flows)
print(change_out_flows)

In [ ]:
change_temp1 = {f"road_{i}_2_3": f"road_{i}_1_3" for i in range(32)}
change_temp2 = {f"road_29_{i}_0": f"road_30_{i}_0" for i in range(32)}
change_temp3 = {f"road_{i}_10_1": f"road_{i}_11_1" for i in range(32)}
change_temp4 = {f"road_2_{i}_2": f"road_1_{i}_2" for i in range(32)}

change_out_dic = {}
change_out_dic.update(change_temp1)
change_out_dic.update(change_temp2)
change_out_dic.update(change_temp3)
change_out_dic.update(change_temp4)

In [ ]:
change_in_dic = {
    "road_30_11_2": "road_31_11_2",
    "road_5_11_3": "road_5_12_3",
    "road_8_1_1": "road_8_0_1",
    "road_22_1_1": "road_22_0_1",
    "road_30_1_2": "road_31_1_2",
    "road_12_1_1": "road_12_0_1",
    "road_13_11_3": "road_13_12_3",
    "road_29_11_3": "road_29_12_3",
    "road_21_11_3": "road_21_12_3",
    "road_28_1_1": "road_28_0_1",
    "road_30_11_3": "road_30_12_3",
    "road_14_1_1": "road_14_0_1",
    "road_26_1_1": "road_26_0_1",
    "road_1_7_0": "road_0_7_0",
    "road_18_1_1": "road_18_0_1",
    "road_11_1_1": "road_11_0_1",
    "roda_9_11_3": "road_9_12_3",
    "road_30_6_2": "road_31_6_2",
    "road_24_1_1": "road_24_0_1",
    "road_11_1_1": "road_11_0_1",
    "road_30_11_2": "road_31_11_2",
    "road_6_1_1": "road_6_0_1",
    "road_1_11_0": "road_0_11_0",
    "road_8_11_3": "road_8_12_3",
    "road_30_3_2": "road_31_3_2",
    "road_22_1_1": "road_22_0_1",
    "road_3_11_3": "road_3_12_3",
    "road_19_11_3": "road_19_12_3",
    "road_1_4_0": "road_0_4_0",
    "road_9_11_3": "road_9_12_3",
    
}

In [ ]:
with open("../Simulation_Results/Manhattan/flow_Penn.json",'r') as f:
    flow_data = json.load(f)
    
    
for flow in flow_data:
    in_flow = flow["route"][0]
    out_flow = flow["route"][-1]
    
    
    if in_flow in change_in_dic:
        flow["route"].insert(0,change_in_dic[in_flow])
    
    if out_flow in change_out_dic:
        flow["route"].append(change_out_dic[out_flow])
        
        

json.dump(flow_data, open("../Simulation_Results/Manhattan/flow_Penn.json", "w"), indent=4)